## 법인 등록번호 가져오기
## ==============
### tool
### 1. selenium
#### $ pip install selenium
### 2. Chrome driver (직접 다운로드)
## ==============
### 크롬 정보 필요 -> chrome://version
###	91.0.4472.114

In [1]:
from selenium import webdriver
# 페이지 불러올 때 까지 기다기기 위함
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

corp = "삼성전자"
browser = webdriver.Chrome('./chromedriver.exe')
# 주소 get
browser.get("http://dart.fss.or.kr/")

# 검색창 element
searchElem = browser.find_element_by_id("textCrpNm")

# 검색창 click
# searchElem.click()

# 키 값 입력을 위함
from selenium.webdriver.common.keys import Keys

# 검색
searchElem.send_keys(corp) # 여기선 위의 import 필요 없음.
searchElem.send_keys(Keys.ENTER)

# 기업 리스트 조회까지 대기
try:
    # 기업 링크 클릭
    corpLink = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.LINK_TEXT, corp)))
    corpLink[0].click()
finally:
    print("공시 System....")

# 법인등록번호 Xpath
xpath = "/html/body/div[8]/div[2]/div[1]/div/div/div/div/div/div/div/div[2]/div/table/tbody/tr[7]/td"
CorpNum = ""
# 기업 정보 팝업이 될 때 까지 대기
try:
    CorpNumber = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    CorpNum = CorpNumber.text
finally: 
    print("정보 조회...")
    browser.quit()

# 사용할 법인 등록 번호
CorpNum = CorpNum[:6]+CorpNum[7:]
print(CorpNum)


공시 System....
정보 조회...
1301110006246


# 기업의 손익계산서, 재무상태표 정보 출력 및 활용
### Ex) 삼성전자
### 법인등록번호 130111-0006246
### 사업자등록번호 124-81-00998
### 회계연도 2020

# ------------------------------
### tool: requests
#### $ python -m pip install requests

### tool: xmltodict
#### $ pip install xmltodict
# ------------------------------
### 참고 : https://www.geeksforgeeks.org/python-xml-to-json/ (XML -> JSON 변환법)


In [2]:
#영미식 단위로 끊어주는 함수

def money(mon):
    a = mon
    result = str(mon)
    finalresult = ""
    count = 0
    while (a > 0):
        a = a//1000
        if(a != 0):
            count+=1
    for i in range(count):
        info = -3*count
        index = info+i*3
        result = result[:index] + ',' + result[index:]
        
    return result
#----------------
#test
money(1320000)
#result = 1,320,000
#----------------
# O(n)

'1,320,000'

In [3]:
import requests as rq
import json, xmltodict

ServiceKey = "ZS%2Fc%2B16anKABmvp0K8KuNoC3%2Fuwb0luEMbFLqCfjFxRuvQGeh2a5VWSgI7acEMi0A27cn%2BRwDQfCOSm5t%2Bs1PQ%3D%3D"

Year = "2020"

#법인등록번호는 위에서 스크래핑 해온 데이터 "CorpNum"사용.
#손익계산서 URL
S_I = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getIncoStat?serviceKey="+ServiceKey+"&numOfRows=10&pageNo=1&numOfRows=xml&crno="+CorpNum+"&bizYear="+Year
#재무상태표 URL
S_FP = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getBs?serviceKey="+ServiceKey+"&numOfRows=18&pageNo=1&numOfRows=xml&crno="+CorpNum+"&bizYear="+Year

### 손익계산서 처리

In [4]:
S_I_content = rq.get(S_I).content
S_I_dict = xmltodict.parse(S_I_content)

In [5]:
#string 형태의 json
S_I_jsonString = json.dumps(S_I_dict['response']['body']['items']['item'], ensure_ascii=False)

In [6]:
#string -> json
S_I_jsonObj = json.loads(S_I_jsonString)

In [7]:
for i in S_I_jsonObj:
    print('====',i['acitNm'],'====')
    print("당분기:", money(int(i['thqrAcitAmt']))+"원")
    print("당기:",money(int(i['crtmAcitAmt']))+"원")
    print("전분기:",money(int(i['lsqtAcitAmt']))+"원")
    print("전기:",money(int(i['pvtrAcitAmt']))+"원")
    print("전전기:",money(int(i['bpvtrAcitAmt']))+"원")
    if(i['fnclDcdNm'] == "연결재무제표 [member]"):
        print("재무제표 종류:","연결재무제표\n")
    elif i['fnclDcdNm'] == "별도재무제표 [member]":
        print("재무제표 종류:","별도재무제표\n")

==== 영업이익 ====
당분기: 0원
당기: 35,993,876,000,000원
전분기: 0원
전기: 27,768,509,000,000원
전전기: 58,886,669,000,000원
재무제표 종류: 연결재무제표

==== 당기순이익(손실) ====
당분기: 0원
당기: 26,407,832,000,000원
전분기: 0원
전기: 21,738,865,000,000원
전전기: 44,344,857,000,000원
재무제표 종류: 연결재무제표

==== 법인세비용차감전순이익(손실) ====
당분기: 0원
당기: 36,345,117,000,000원
전분기: 0원
전기: 30,432,189,000,000원
전전기: 61,159,958,000,000원
재무제표 종류: 연결재무제표

==== 수익(매출액) ====
당분기: 0원
당기: 236,806,988,000,000원
전분기: 0원
전기: 230,400,881,000,000원
전전기: 243,771,415,000,000원
재무제표 종류: 연결재무제표

==== 영업이익 ====
당분기: 0원
당기: 20,518,974,000,000원
전분기: 0원
전기: 14,115,067,000,000원
전전기: 43,699,451,000,000원
재무제표 종류: 별도재무제표

==== 당기순이익(손실) ====
당분기: 0원
당기: 15,615,018,000,000원
전분기: 0원
전기: 15,353,323,000,000원
전전기: 32,815,127,000,000원
재무제표 종류: 별도재무제표

==== 법인세비용차감전순이익(손실) ====
당분기: 0원
당기: 20,451,923,000,000원
전분기: 0원
전기: 19,032,469,000,000원
전전기: 44,398,855,000,000원
재무제표 종류: 별도재무제표

==== 수익(매출액) ====
당분기: 0원
당기: 166,311,191,000,000원
전분기: 0원
전기: 154,772,859,000,000원
전전기: 170,381,870,000,000원
재무제표 

### 재무상태표 처리

In [8]:
S_FP_content = rq.get(S_FP).content
S_FP_dict = xmltodict.parse(S_FP_content)

In [9]:
#string 형태의 json
S_FP_jsonString = json.dumps(S_FP_dict['response']['body']['items']['item'], ensure_ascii=False)

In [10]:
#string -> json
S_FP_jsonObj = json.loads(S_FP_jsonString)

In [11]:
for i in S_FP_jsonObj:
    print('====',i['acitNm'],'====')
    print("당분기:", money(int(i['thqrAcitAmt']))+"원")
    print("당기:",money(int(i['crtmAcitAmt']))+"원")
    print("전분기:",money(int(i['lsqtAcitAmt']))+"원")
    print("전기:",money(int(i['pvtrAcitAmt']))+"원")
    print("전전기:",money(int(i['bpvtrAcitAmt']))+"원")
    if(i['fnclDcdNm'] == "연결재무제표 [member]"):
        print("재무제표 종류:","연결재무제표\n")
    elif i['fnclDcdNm'] == "별도재무제표 [member]":
        print("재무제표 종류:","별도재무제표\n")

==== 자산총계 ====
당분기: 0원
당기: 378,235,718,000,000원
전분기: 0원
전기: 352,564,497,000,000원
전전기: 339,357,244,000,000원
재무제표 종류: 연결재무제표

==== 유동자산 ====
당분기: 0원
당기: 198,215,579,000,000원
전분기: 0원
전기: 181,385,260,000,000원
전전기: 174,697,424,000,000원
재무제표 종류: 연결재무제표

==== 유동부채 ====
당분기: 0원
당기: 75,604,351,000,000원
전분기: 0원
전기: 63,782,764,000,000원
전전기: 69,081,510,000,000원
재무제표 종류: 연결재무제표

==== 자본총계 ====
당분기: 0원
당기: 275,948,016,000,000원
전분기: 0원
전기: 262,880,421,000,000원
전전기: 247,753,177,000,000원
재무제표 종류: 연결재무제표

==== 자본금 ====
당분기: 0원
당기: 897,514,000,000원
전분기: 0원
전기: 897,514,000,000원
전전기: 897,514,000,000원
재무제표 종류: 연결재무제표

==== 부채총계 ====
당분기: 0원
당기: 102,287,702,000,000원
전분기: 0원
전기: 89,684,076,000,000원
전전기: 91,604,067,000,000원
재무제표 종류: 연결재무제표

==== 비유동자산 ====
당분기: 0원
당기: 180,020,139,000,000원
전분기: 0원
전기: 171,179,237,000,000원
전전기: 164,659,820,000,000원
재무제표 종류: 연결재무제표

==== 비유동부채 ====
당분기: 0원
당기: 26,683,351,000,000원
전분기: 0원
전기: 25,901,312,000,000원
전전기: 22,522,557,000,000원
재무제표 종류: 연결재무제표

==== 이익잉여금(결손금) ====
당분기: 0